In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

In [2]:
all_data_path = feature_data_path + 'all_data_all_features.pkl'
all_data = load_pickle(all_data_path)

# all_data.drop(['user_click_rank_day', 'user_first_click_day', 'user_last_click_day'], axis=1, inplace=True)
# all_data.drop(['user_click_interval_first_day', 'user_click_interval_last_day'], axis=1, inplace=True)


train_data = all_data[(all_data.day >= 19) & (all_data.day <= 23)]
test_data = all_data[all_data.day == 24]

features = list(all_data.columns)
features.remove('is_trade')
features.remove('instance_id')
target = ['is_trade']
len(features)

332

In [3]:
features

['item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_id',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_id',
 'context_timestamp',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'day',
 'hour',
 'minute',
 'category2_label',
 'item_property_list0',
 'item_property_list1',
 'item_property_list2',
 'item_property_list3',
 'item_property_list4',
 'item_property_list5',
 'item_property_list6',
 'item_property_list7',
 'user_item_id_click_day',
 'user_item_brand_id_click_day',
 'user_item_city_id_click_day',
 'user_category2_label_click_day',
 'user_item_price_level_click_day',
 'user_item_sales_level_click_day',
 'user_item_collected_level_click_day',
 'user_item_pv_level_click_day',
 'user_context_page_id_click_day',
 'user_sh

In [6]:
features=[ 
#预处理后的基础特征----------------------------------------------
 'item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_id',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_id',
 'context_timestamp',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'day',
 'hour',
 'minute',
 'category2_label',
 'item_property_list0',
 'item_property_list1',
 'item_property_list2',
 'item_property_list3',
 'item_property_list4',
 'item_property_list5',
 'item_property_list6',
 'item_property_list7',
    
#  2_1处理后的特征
#  生成用户对当天属性的点击量 ['item_id', 'item_brand_id', 'item_city_id', 'context_page_id', 'shop_id', 'category2_label',
#                 'item_sales_level', 'item_price_level', 'item_collected_level', 'item_pv_level']   
 'user_item_id_click_day',
 'user_item_brand_id_click_day',
 'user_item_city_id_click_day',
 'user_context_page_id_click_day',
 'user_shop_id_click_day',
 'user_category2_label_click_day',
 'user_item_sales_level_click_day',
 'user_item_price_level_click_day',
 'user_item_collected_level_click_day',
 'user_item_pv_level_click_day',
#  2_1处理后的特征
#  生成用户对当天当小时属性的点击量 ['item_id', 'item_brand_id', 'item_city_id', 'context_page_id', 'shop_id', 'category2_label',
#                 'item_sales_level', 'item_price_level', 'item_collected_level', 'item_pv_level']   
 'user_item_id_click_hour',
 'user_item_brand_id_click_hour',
 'user_item_city_id_click_hour',
 'user_context_page_id_click_hour',
 'user_shop_id_click_hour',
 'user_category2_label_click_hour',
 'user_item_sales_level_click_hour',
 'user_item_price_level_click_hour',
 'user_item_collected_level_click_hour',
 'user_item_pv_level_click_hour',
#  2_1处理后的特征
#  生成用户对单一特征点击数据的统计特征 
 'user_item_id_click_day_mean',
 'user_item_id_click_day_min',
 'user_item_id_click_day_max',
 'user_item_brand_id_click_day_mean',
 'user_item_brand_id_click_day_min',
 'user_item_brand_id_click_day_max',
 'user_shop_id_click_day_mean',
 'user_shop_id_click_day_min',
 'user_shop_id_click_day_max',
 'user_category2_label_click_day_mean',
 'user_category2_label_click_day_min',
 'user_category2_label_click_day_max',
    
    
#  2_2处理后的特征
#  生成单一特征，日点击量的统计特征    stats_feature = ['user_id', 'item_id', 'item_brand_id', 'shop_id']   
 'user_id_click_day_mean',
 'user_id_click_day_max',
 'user_id_click_day_min',
 'item_id_click_day_mean',
 'item_id_click_day_max',
 'item_id_click_day_min',
 'item_brand_id_click_day_mean',
 'item_brand_id_click_day_max',
 'item_brand_id_click_day_min',
 'shop_id_click_day_mean',
 'shop_id_click_day_max',
 'shop_id_click_day_min',
    
    
#  2_3处理后的特征
#  生成用户日点击时间差特征     
 'user_click_rank_day',
 'user_first_click_day',
 'user_last_click_day',
 'user_click_interval_first_day',
 'user_click_interval_last_day',
 'user_click_interval_diff_day',
 'user_click_interval_prob',
 'time_gap_before',
 'time_gap_after',
#  2_3处理后的特征
#  生成用户对属性日点击时间差特征   feature_list = ['item_id', 'item_brand_id', 'item_city_id', 'shop_id', 'context_page_id', 'category2_label',
#                   'item_sales_level', 'item_price_level', 'item_collected_level', 'item_pv_level']
##################################    这里我想进一步删减一些，太多了   #######################################
 'user_item_id_first_click',
 'user_item_id_last_click',
 'user_item_id_click_rank',
 'user_item_id_first_click_interval',
 'user_item_id_last_click_interval',
 'user_item_id_diff_click_interval',
 'user_item_id_prob_click_interval',
 'item_id_time_gap_before',
 'item_id_time_gap_after',
 'user_item_brand_id_first_click',
 'user_item_brand_id_last_click',
 'user_item_brand_id_click_rank',
 'user_item_brand_id_first_click_interval',
 'user_item_brand_id_last_click_interval',
 'user_item_brand_id_diff_click_interval',
 'user_item_brand_id_prob_click_interval',
 'item_brand_id_time_gap_before',
 'item_brand_id_time_gap_after',
#  'user_item_city_id_first_click',
#  'user_item_city_id_last_click',
#  'user_item_city_id_click_rank',
#  'user_item_city_id_first_click_interval',
#  'user_item_city_id_last_click_interval',
#  'user_item_city_id_diff_click_interval',
#  'user_item_city_id_prob_click_interval',
#  'item_city_id_time_gap_before',
#  'item_city_id_time_gap_after',
 'user_shop_id_first_click',
 'user_shop_id_last_click',
 'user_shop_id_click_rank',
 'user_shop_id_first_click_interval',
 'user_shop_id_last_click_interval',
 'user_shop_id_diff_click_interval',
 'user_shop_id_prob_click_interval',
 'shop_id_time_gap_before',
 'shop_id_time_gap_after',
 'user_context_page_id_first_click',
 'user_context_page_id_last_click',
 'user_context_page_id_click_rank',
 'user_context_page_id_first_click_interval',
 'user_context_page_id_last_click_interval',
 'user_context_page_id_diff_click_interval',
 'user_context_page_id_prob_click_interval',
 'context_page_id_time_gap_before',
 'context_page_id_time_gap_after',
 'user_category2_label_first_click',
 'user_category2_label_last_click',
 'user_category2_label_click_rank',
 'user_category2_label_first_click_interval',
 'user_category2_label_last_click_interval',
 'user_category2_label_diff_click_interval',
 'user_category2_label_prob_click_interval',
 'category2_label_time_gap_before',
 'category2_label_time_gap_after',
#  'user_item_sales_level_first_click',
#  'user_item_sales_level_last_click',
#  'user_item_sales_level_click_rank',
#  'user_item_sales_level_first_click_interval',
#  'user_item_sales_level_last_click_interval',
#  'user_item_sales_level_diff_click_interval',
#  'user_item_sales_level_prob_click_interval',
#  'item_sales_level_time_gap_before',
#  'item_sales_level_time_gap_after',
#  'user_item_price_level_first_click',
#  'user_item_price_level_last_click',
#  'user_item_price_level_click_rank',
#  'user_item_price_level_first_click_interval',
#  'user_item_price_level_last_click_interval',
#  'user_item_price_level_diff_click_interval',
#  'user_item_price_level_prob_click_interval',
#  'item_price_level_time_gap_before',
#  'item_price_level_time_gap_after',
#  'user_item_collected_level_first_click',
#  'user_item_collected_level_last_click',
#  'user_item_collected_level_click_rank',
#  'user_item_collected_level_first_click_interval',
#  'user_item_collected_level_last_click_interval',
#  'user_item_collected_level_diff_click_interval',
#  'user_item_collected_level_prob_click_interval',
#  'item_collected_level_time_gap_before',
#  'item_collected_level_time_gap_after',
#  'user_item_pv_level_first_click',
#  'user_item_pv_level_last_click',
#  'user_item_pv_level_click_rank',
#  'user_item_pv_level_first_click_interval',
#  'user_item_pv_level_last_click_interval',
#  'user_item_pv_level_diff_click_interval',
#  'user_item_pv_level_prob_click_interval',
#  'item_pv_level_time_gap_before',
#  'item_pv_level_time_gap_after',
    
#  2_4处理后的特征
 'category_predict_rank',

    
#  2_5处理后的特征
#  生成历史点击率，要去除点击次数和点击时间
#    ['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level',
#                         'category_predict_rank', 'item_sales_level', 'item_collected_level', 'item_pv_level']
###################################### 我认为 这里还要加上一些用户的星级，职业等###########################################    
#  'user_id_smooth_I',
#  'user_id_smooth_C',
 'user_id_smooth_CTR',
#  'item_id_smooth_I',
#  'item_id_smooth_C',
 'item_id_smooth_CTR',
#  'item_brand_id_smooth_I',
#  'item_brand_id_smooth_C',
 'item_brand_id_smooth_CTR',
#  'shop_id_smooth_I',
#  'shop_id_smooth_C',
 'shop_id_smooth_CTR',
#  'context_page_id_smooth_I',
#  'context_page_id_smooth_C',
 'context_page_id_smooth_CTR',
#  'category2_label_smooth_I',
#  'category2_label_smooth_C',
 'category2_label_smooth_CTR',
#  'item_price_level_smooth_I',
#  'item_price_level_smooth_C',
 'item_price_level_smooth_CTR',
#  'category_predict_rank_smooth_I',
#  'category_predict_rank_smooth_C',
 'category_predict_rank_smooth_CTR',
#  'item_sales_level_smooth_I',
#  'item_sales_level_smooth_C',
 'item_sales_level_smooth_CTR',
#  'item_collected_level_smooth_I',
#  'item_collected_level_smooth_C',
 'item_collected_level_smooth_CTR',
#  'item_pv_level_smooth_I',
#  'item_pv_level_smooth_C',
 'item_pv_level_smooth_CTR',
    
#  2_5处理后的特征
#  生成前一天点击率，前一天的点击次数，前一天的购买次数
#    ['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level',
#                         'category_predict_rank', 'item_sales_level', 'item_collected_level', 'item_pv_level']
###################################### 我认为 这里还要加上一些用户的星级，职业等###########################################  
 'user_id_day_I',
 'user_id_day_C',
 'user_id_CTR',
 'item_id_day_I',
 'item_id_day_C',
 'item_id_CTR',
 'item_brand_id_day_I',
 'item_brand_id_day_C',
 'item_brand_id_CTR',
 'shop_id_day_I',
 'shop_id_day_C',
 'shop_id_CTR',
 'context_page_id_day_I',
 'context_page_id_day_C',
 'context_page_id_CTR',
 'category2_label_day_I',
 'category2_label_day_C',
 'category2_label_CTR',
 'item_price_level_day_I',
 'item_price_level_day_C',
 'item_price_level_CTR',
 'category_predict_rank_day_I',
 'category_predict_rank_day_C',
 'category_predict_rank_CTR',
 'item_sales_level_day_I',
 'item_sales_level_day_C',
 'item_sales_level_CTR',
 'item_collected_level_day_I',
 'item_collected_level_day_C',
 'item_collected_level_CTR',
 'item_pv_level_day_I',
 'item_pv_level_day_C',
 'item_pv_level_CTR',
    
#  2_5处理后的特征
#  生成前一天交叉点击率，前一天的点击次数，前一天的购买次数
#    ['user_gender_id', 'user_age_level', 'user_occupation_id']
#   ['item_id', 'item_brand_id', 'shop_id']
###################################### 这里面历史点击率还没做好###########################################  
 'user_gender_id_item_id_day_I',
 'user_gender_id_item_id_day_C',
 'user_gender_id_item_id_day_CTR',
 'user_gender_id_item_brand_id_day_I',
 'user_gender_id_item_brand_id_day_C',
 'user_gender_id_item_brand_id_day_CTR',
 'user_gender_id_shop_id_day_I',
 'user_gender_id_shop_id_day_C',
 'user_gender_id_shop_id_day_CTR',
 'user_age_level_item_id_day_I',
 'user_age_level_item_id_day_C',
 'user_age_level_item_id_day_CTR',
 'user_age_level_item_brand_id_day_I',
 'user_age_level_item_brand_id_day_C',
 'user_age_level_item_brand_id_day_CTR',
 'user_age_level_shop_id_day_I',
 'user_age_level_shop_id_day_C',
 'user_age_level_shop_id_day_CTR',
 'user_occupation_id_item_id_day_I',
 'user_occupation_id_item_id_day_C',
 'user_occupation_id_item_id_day_CTR',
 'user_occupation_id_item_brand_id_day_I',
 'user_occupation_id_item_brand_id_day_C',
 'user_occupation_id_item_brand_id_day_CTR',
 'user_occupation_id_shop_id_day_I',
 'user_occupation_id_shop_id_day_C',
 'user_occupation_id_shop_id_day_CTR',
    
#  2_6处理后的特征
#  分别groupby['shop_id'], ['item_id'], ['item_brand_id']
#    计算item在['user_gender_id', 'user_age_level', 'user_occupation_id']几个属性下的点击量
 'shop_id_user_gender_id_click_rate',
 'shop_id_user_age_level_click_rate',
 'shop_id_user_occupation_id_click_rate',
 'shop_id_user_star_level_click_rate',
 'item_id_user_gender_id_click_rate',
 'item_id_user_age_level_click_rate',
 'item_id_user_occupation_id_click_rate',
 'item_id_user_star_level_click_rate',
 'item_brand_id_user_gender_id_click_rate',
 'item_brand_id_user_age_level_click_rate',
 'item_brand_id_user_occupation_id_click_rate',
 'item_brand_id_user_star_level_click_rate',
    
#  2_7处理后的特征 
# ['user_id', 'item_id', 'item_brand_id', 'shop_id', 'user_gender_id', 'context_page_id',
#                         'user_occupation_id', 'user_age_level', 'category2_label', 'item_sales_level', 'item_price_level', 'user_star_level',
#                         'item_collected_level', 'item_pv_level']  
#  计算每天的点击量
########################## 我认为，这里应该只留下['user_id', 'item_id', 'item_brand_id', 'shop_id']################################
 'user_id_click_day',
 'item_id_click_day',
 'item_brand_id_click_day',
 'shop_id_click_day',
 'user_gender_id_click_day',
 'context_page_id_click_day',
 'user_occupation_id_click_day',
 'user_age_level_click_day',
 'category2_label_click_day',
 'item_sales_level_click_day',
 'item_price_level_click_day',
 'user_star_level_click_day',
 'item_collected_level_click_day',
 'item_pv_level_click_day',
#  2_7处理后的特征
#  计算每天每小时的点击量 
########################## 我认为，这里应该只留下['user_id', 'item_id', 'item_brand_id', 'shop_id', 'user_gender_id',
#                         'user_occupation_id', 'user_age_level', ]
 'user_id_click_hour_x',
 'item_id_click_hour_x',
 'item_brand_id_click_hour_x',
 'shop_id_click_hour_x',
 'user_gender_id_click_hour_x',
 'context_page_id_click_hour_x',
 'user_occupation_id_click_hour_x',
 'user_age_level_click_hour_x',
 'category2_label_click_hour_x',
 'item_sales_level_click_hour_x',
 'item_price_level_click_hour_x',
 'user_star_level_click_hour_x',
 'item_collected_level_click_hour_x',
 'item_pv_level_click_hour_x',
#  2_7处理后的特征
#  计算每小时的点击量
########################## 我认为，这里应该只留下['user_id', 'item_id', 'item_brand_id', 'shop_id', 'user_gender_id',
#                         'user_occupation_id', 'user_age_level', ]
 'user_id_click_hour_y',
 'item_id_click_hour_y',
 'item_brand_id_click_hour_y',
 'shop_id_click_hour_y',
 'user_gender_id_click_hour_y',
 'context_page_id_click_hour_y',
 'user_occupation_id_click_hour_y',
 'user_age_level_click_hour_y',
 'category2_label_click_hour_y',
 'item_sales_level_click_hour_y',
 'item_price_level_click_hour_y',
 'user_star_level_click_hour_y',
 'item_collected_level_click_hour_y',
 'item_pv_level_click_hour_y', 
]

In [7]:
all_data[features].shape
# features

(496482, 258)

In [10]:
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
import lightgbm as lgb

lgb_train = lgb.Dataset(train_data[features], train_data['is_trade'])
lgb_eval = lgb.Dataset(test_data[features], test_data['is_trade'], reference=lgb_train)
cate_features = ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    #'num_leaves': 31,
    'learning_rate': 0.05,
     'feature_fraction': 0.9,
     'bagging_fraction': 0.9,
     'bagging_freq': 5,
     'verbose': 0,
    #'lambda_l2':10,
    'max_depth':4
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=[lgb_eval, lgb_train],
                valid_names = ['eval', 'train'] ,
                categorical_feature=cate_features,
                early_stopping_rounds=20,
                verbose_eval = 20)

loss_train = log_loss(train_data[target], gbm.predict(train_data[features]))
loss_test = log_loss(test_data[target], gbm.predict(test_data[features]))

loss_train, loss_test

Start training...


D:\Program Files (x86)\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_age_level', 'user_gender_id', 'user_occupation_id', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Program Files (x86)\Anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 20 rounds.
[20]	train's binary_logloss: 0.240957	eval's binary_logloss: 0.238152
[40]	train's binary_logloss: 0.129175	eval's binary_logloss: 0.124667
[60]	train's binary_logloss: 0.0974441	eval's binary_logloss: 0.0919523
[80]	train's binary_logloss: 0.0887683	eval's binary_logloss: 0.082946
[100]	train's binary_logloss: 0.0861314	eval's binary_logloss: 0.0804135
[120]	train's binary_logloss: 0.0849274	eval's binary_logloss: 0.0795797
[140]	train's binary_logloss: 0.0840745	eval's binary_logloss: 0.0791597
[160]	train's binary_logloss: 0.0833853	eval's binary_logloss: 0.0789545
[180]	train's binary_logloss: 0.0827763	eval's binary_logloss: 0.0787918
[200]	train's binary_logloss: 0.0821977	eval's binary_logloss: 0.07873
[220]	train's binary_logloss: 0.0816602	eval's binary_logloss: 0.0786698
[240]	train's binary_logloss: 0.0811641	eval's binary_logloss: 0.0786756
[260]	train's binary_logloss: 0.0807102	eval's binary_logloss: 0.0786081


(0.08092683590201565, 0.07860720812703091)

In [7]:
importance = pd.DataFrame(gbm.feature_importance())
importance.columns = ['importance_20']
features = pd.DataFrame(features)
features.columns = ['features_20']
pd.set_option('max_rows',500)
merge = features.join(importance)
merge = merge.sort_values(by=['importance_20'], ascending=False).reset_index()
merge

,index,features_20,importance_20
0,134,item_id_smooth_CTR,698
1,86,time_gap_after,670
2,136,shop_id_smooth_CTR,596
3,127,user_category2_label_last_click_interval,591
4,20,shop_score_delivery,500
5,23,hour,419
6,135,item_brand_id_smooth_CTR,349
7,89,user_item_id_click_rank,324
8,9,user_age_level,322
9,11,user_star_level,297


In [ ]:
dump_pickle(merge, feature_data_path+'importance_20.pkl')

In [ ]:
merge24 = load_pickle(feature_data_path+'importance_24.pkl')
merge24 = merge24.rename(columns={'index':'index24'})
merge23 = load_pickle(feature_data_path+'importance_23.pkl')
merge23 = merge23.rename(columns={'index':'index23'})
merge22 = load_pickle(feature_data_path+'importance_22.pkl')
merge22 = merge22.rename(columns={'index':'index22'})
merge20 = load_pickle(feature_data_path+'importance_20.pkl')
merge20 = merge20.rename(columns={'index':'index20'})
merge = merge24.join(merge23)
merge = merge.join(merge22)
merge = merge.join(merge20)

writer = pd.ExcelWriter('merge.xlsx')
merge.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', max_depth=5, learning_rate=0.01,
                        n_estimators=3000, objective='binary', 
                        subsample=0.9, subsample_freq=5, colsample_bytree=0.9, 
                         n_jobs=-1, silent=True)
cate_features = ['user_gender_id', 'user_occupation_id']
lgb_clf.fit(train_data[features], train_data['is_trade'],
          eval_set=[(test_data[features],test_data[target])],
          early_stopping_rounds=20,
          feature_name=features,
          categorical_feature=cate_features,
          verbose=20,
          )
loss_train = log_loss(train_data[target], lgb_clf.predict_proba(train_data[features]))
loss_test = log_loss(test_data[target], lgb_clf.predict_proba(test_data[features]))

loss_train, loss_test

\## 提交结果

In [5]:
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
import lightgbm as lgb

train_data = all_data[(all_data.day >= 19) & (all_data.day <= 24)]
test_data = all_data[all_data.day == 25]

lgb_train = lgb.Dataset(train_data[features], train_data['is_trade'])
lgb_test = lgb.Dataset(test_data[features], test_data['is_trade'], reference=lgb_train)
cate_features = ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    #'num_leaves': 31,
    'learning_rate': 0.01,
     'feature_fraction': 0.9,
     'bagging_fraction': 0.9,
     'bagging_freq': 5,
     'verbose': 0,
    'lambda_l2':10,
    'max_depth':4
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1100,
                valid_sets=[lgb_train],
                valid_names = ['train'] ,
                categorical_feature=cate_features,
                early_stopping_rounds=20,
                verbose_eval = 5)

loss_train = log_loss(train_data[target], gbm.predict(train_data[features]))

test_data['predicted_score'] = gbm.predict(test_data[features])

test_data[['instance_id', 'predicted_score']].to_csv('20180412.txt', index=False, sep=' ')

loss_train

Start training...


D:\Program Files (x86)\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_age_level', 'user_gender_id', 'user_occupation_id', 'user_star_level']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 20 rounds.
[5]	train's binary_logloss: 0.648792
[10]	train's binary_logloss: 0.608513
[15]	train's binary_logloss: 0.571804
[20]	train's binary_logloss: 0.538213
[25]	train's binary_logloss: 0.507406
[30]	train's binary_logloss: 0.479074
[35]	train's binary_logloss: 0.452946
[40]	train's binary_logloss: 0.428813
[45]	train's binary_logloss: 0.406482
[50]	train's binary_logloss: 0.385779
[55]	train's binary_logloss: 0.366554
[60]	train's binary_logloss: 0.34868
[65]	train's binary_logloss: 0.332043
[70]	train's binary_logloss: 0.316544
[75]	train's binary_logloss: 0.302079
[80]	train's binary_logloss: 0.288572
[85]	train's binary_logloss: 0.27596
[90]	train's binary_logloss: 0.264161
[95]	train's binary_logloss: 0.253128
[100]	train's binary_logloss: 0.242792
[105]	train's binary_logloss: 0.233106
[110]	train's binary_logloss: 0.224039
[115]	train's binary_logloss: 0.215527
[120]	train's binary_logloss: 0.207547
[125]	train's binary_log

D:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.08201450552613691

# 提交结果先用这个

In [6]:
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
import lightgbm as lgb
train_data = all_data[(all_data.day >= 19) & (all_data.day <= 24)]
print(train_data.shape)
test_data = all_data[all_data.day == 25]
cate_features = ['user_gender_id', 'user_occupation_id',]
clf = LGBMClassifier(n_estimators=200, max_depth=3)
                     
clf.fit(train_data[features], train_data['is_trade'],
          feature_name=features,
          categorical_feature=cate_features,
          )

loss_train = log_loss(train_data[target], clf.predict_proba(train_data[features]))

test_data['predicted_score'] = clf.predict_proba(test_data[features])[:, 1]

test_data[['instance_id', 'predicted_score']].to_csv(
    '20180410.txt', index=False, sep=' ')

loss_train

(399850, 328)


D:\Program Files (x86)\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_gender_id', 'user_occupation_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
D:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.08137898505627898